# Large-scale Text Processing in Core Python

Portfolio notebook demonstrating efficient processing of large text datasets using core Python only (no Pandas/NumPy/NLP libraries).  
Focus: streaming I/O, memory-aware data structures, and algorithmic text analysis.


## What’s inside
- Streaming and parsing line-delimited JSON datasets (Wikipedia-style articles + tweets)
- Text normalisation/token handling
- Frequency analysis and case-variant tracking
- Similarity matching using Jaccard index (with pruning)
- Building an inverted index and supporting wildcard-style search


## Data
This notebook expects the following files to be available locally in a `data/` directory (not committed to GitHub due to size and licensing constraints):

- `wiki-articles.json`
- `tweets.json`
- `linuxwords`


## Notebook structure
1. Wikipedia corpus analysis
2. Twitter corpus analysis
3. Cross-corpus matching (Jaccard similarity)
4. Inverted index + wildcard search


In [1]:
# Setup & utilities
# - Resolve data directory for both university JupyterHub and GitHub layouts
# - Define shared text-processing helpers used throughout the notebook

from pathlib import Path
import json
import string
from collections import Counter, defaultdict

# Resolve data directory
DATA_DIR = Path.cwd()
if not (DATA_DIR / "wiki-articles.json").exists():
    DATA_DIR = Path.cwd() / "data"

# Note: this translator removes all punctuation.
# Twitter-specific processing (e.g. @mentions) is handled separately where needed.
TRANSLATOR = str.maketrans("", "", string.punctuation)

def normalise(text: str) -> list[str]:
    """Lowercase, strip punctuation, and split on whitespace."""
    return text.translate(TRANSLATOR).lower().split()

def tokens_from_json_line(obj: dict, field: str) -> list[str]:
    """Extract and normalise tokens from a JSON object field."""
    value = obj.get(field, "")
    return normalise(value) if isinstance(value, str) else []


**Note**: Tokenisation here is intentionally simple (whitespace-based, punctuation stripped) to keep the focus on algorithmic text processing rather than linguistic accuracy.


## 1) Wikipedia: Most frequent unique words

### Objective
Identify the most frequently occurring words across the Wikipedia article corpus, counting each word at most once per article.

This approach avoids overweighting articles that repeat the same term many times and instead highlights words that appear broadly across the corpus.


In [3]:
# 1) Wikipedia: Most frequent unique words

# Count of unique word occurrences across articles
word_counts = Counter()

with open(DATA_DIR / "wiki-articles.json", "r", encoding="utf-8") as fh:
    for line in fh:
        article = json.loads(line)
        text = article.get("text", "")
        unique_words = set(normalise(text))  # count each token at most once per article
        word_counts.update(unique_words)

# Filter very common stopwords for a more informative top list
STOPWORDS = {"the", "and", "of", "to", "in", "a", "is", "for", "on", "with", "what"}

filtered_counts = Counter(
    {word: count for word, count in word_counts.items() if word not in STOPWORDS and len(word) > 2}
)

print("Top 10 most frequent words across articles (unique per article):")
for word, count in filtered_counts.most_common(10):
    print(f"{word:<15} {count}")


Top 10 most frequent words across articles (unique per article):
was             17131
from            16460
that            16328
this            16173
which           16027
also            15970
are             15312
has             15159
one             14850
other           14828


## 2) Wikipedia: Longest capitalised phrase

### Objective
Identify the longest contiguous sequence of capitalised words appearing in Wikipedia article body text, where a word is considered capitalised if its first character is uppercase.

For example, in the sentence  
“Queen Mary University is located in the United Kingdom, specifically in London”,  
the capitalised sequences are “Queen Mary University”, “United Kingdom”, and “London”.  
The longest sequence in this case is “Queen Mary University”.


In [4]:
# 2) Wikipedia: Longest capitalised phrase

def is_capitalised_token(token: str) -> bool:
    """
    A token is 'capitalised' if its first character is an uppercase letter.
    We also ignore tokens that start with a digit or punctuation (after cleaning).
    """
    return bool(token) and token[0].isalpha() and token[0].isupper()

def longest_capitalised_sequence(tokens: list[str]) -> list[str]:
    """Return the longest contiguous subsequence of capitalised tokens."""
    best_start = best_len = 0
    cur_start = cur_len = 0

    for i, tok in enumerate(tokens):
        if is_capitalised_token(tok):
            if cur_len == 0:
                cur_start = i
            cur_len += 1

            if cur_len > best_len:
                best_len = cur_len
                best_start = cur_start
        else:
            cur_len = 0

    return tokens[best_start:best_start + best_len]

best_sequence = []
best_article_title = None

with open(DATA_DIR / "wiki-articles.json", "r", encoding="utf-8") as fh:
    for line in fh:
        article = json.loads(line)

        # Keep original case (do NOT call normalise here, as it lowercases)
        text = article.get("text", "")
        tokens = text.translate(TRANSLATOR).split()

        seq = longest_capitalised_sequence(tokens)
        if len(seq) > len(best_sequence):
            best_sequence = seq
            best_article_title = article.get("title")

# Reduce output spam: show preview + length + where it was found
preview_n = 25
preview = " ".join(best_sequence[:preview_n])
suffix = " ..." if len(best_sequence) > preview_n else ""

print(f"Longest sequence length: {len(best_sequence)} words")
print(f"Found in article: {best_article_title}")
print(f"Preview: {preview}{suffix}")


Longest sequence length: 134 words
Found in article: Humphrey Bogart
Preview: Havilland Michael Curtiz James Cagney David O Selznick William Wyler Richard Brooks Harry Cohn Jane Wyman Jean Arthur Claude Rains Raymond Massey George Raft Myrna ...


## 3) Wikipedia: Largest anagram set (filtered)

### Objective
Identify the largest set of anagrammatic words appearing in Wikipedia article body text, considering only words with at least six characters.

Two words are anagrams if they consist of the same characters in a different order (including repeated characters). For example:
- “cheap” and “peach”
- “reacting” and “creating”
- “resistance” and “ancestries”

Anagram sets may contain more than two words. For instance:
- {“asleep”, “please”, “elapse”} form a 3-word anagram set
- {“aspired”, “despair”, “diapers”, “praised”} form a 4-word anagram set

Words are collected across the entire article corpus (not limited to a single article), and only article body text is considered.


In [5]:
# 3) Wikipedia: Largest anagram set (filtered)

def signature(word: str) -> str:
    """Canonical anagram signature: sorted characters."""
    return "".join(sorted(word))

groups = defaultdict(set)

with open(DATA_DIR / "wiki-articles.json", "r", encoding="utf-8") as fh:
    for line in fh:
        article = json.loads(line)
        text = article.get("text", "")

        for w in normalise(text):
            # Filters to avoid tokenisation artefacts and "non-words"
            # - length constraint from objective (>= 6)
            # - alphabetic only (removes things like abcdef..., mixed tokens, etc.)
            # - reasonable max length to avoid weird long strings dominating
            if 6 <= len(w) <= 20 and w.isalpha():
                groups[signature(w)].add(w)

# Find the largest anagram group
best_sig, best_group = max(groups.items(), key=lambda kv: len(kv[1]))

best_sorted = sorted(best_group)

print(f"Largest anagram set size: {len(best_sorted)}")
print(f"Example signature: {best_sig}")
print("Words (showing up to 20):")
for word in best_sorted[:20]:
    print(f"- {word}")

if len(best_sorted) > 20:
    print(f"... (+{len(best_sorted) - 20} more)")


Largest anagram set size: 19
Example signature: aeimnr
Words (showing up to 20):
- airmen
- armeni
- ermina
- mainer
- maneri
- manier
- marien
- marine
- marnie
- meirna
- merian
- merina
- minera
- mirena
- namier
- nerima
- reiman
- remain
- rmaine


## 4) Twitter: Top mentioned users

### Objective
Identify the most frequently mentioned Twitter users by analysing tokens that begin with the `@` symbol, and report the top five users along with their mention counts.

During tokenisation, punctuation is removed while preserving the `@` character so that user mentions are not discarded. While this approach is not perfect in all edge cases, it is sufficient for reliably extracting mentions from the dataset.


In [6]:
# 4) Twitter: Top mentioned users

# Create a translation table that removes punctuation EXCEPT '@'
MENTION_TRANSLATOR = str.maketrans("", "", string.punctuation.replace("@", ""))

mentions_by_user = Counter()

with open(DATA_DIR / "tweets.json", "r", encoding="utf-8") as fh:
    for line in fh:
        tweet_obj = json.loads(line)
        tweet_text = tweet_obj.get("text", "")

        # Remove punctuation (except '@') and lowercase for consistent counting
        cleaned = tweet_text.translate(MENTION_TRANSLATOR).lower()

        # Extract @mentions and strip the '@'
        for token in cleaned.split():
            if token.startswith("@") and len(token) > 1:
                username = token[1:]
                # Basic sanity filter: usernames should be alphanumeric/underscore
                if username.replace("_", "").isalnum():
                    mentions_by_user[username] += 1

print("Top 5 mentioned users:")
for username, count in mentions_by_user.most_common(5):
    print(f"@{username:<15} {count}")


Top 5 mentioned users:
@btstwt          31880
@enhypenmembers  4702
@enhypen         4169
@blackpink       3891
@nctsmtown       3450


## 5) Twitter: Word with most case variations

### Objective
Identify the word in the Twitter dataset that appears with the largest number of distinct case variations, while preserving the original casing of tokens.

Case variations refer to tokens that differ only by letter casing (e.g. “house” vs “HouSe”) and are treated as the same underlying word when lowercased, but counted separately based on their original form.

For example, given the tokens  
“HouSe, House, HOUSE, YES, yes, YeS, Yes, yeS, yEs”,  
the word “house” has 3 case variations and “yes” has 6.  
In this example, “yes” would be identified as the word with the most case variations.


In [7]:
# 5) Twitter: Word with most case variations

# exclude very common stopwords so results are more informative
STOPWORDS = {"the", "and", "of", "to", "in", "a", "is", "for", "on", "with", "what"}

def iter_alpha_tokens_preserve_case(text: str):
    """
    Yield alphabetic tokens while preserving original case.
    Punctuation is stripped from token boundaries only.
    """
    for raw in text.split():
        token = raw.strip(string.punctuation)
        if token.isalpha():
            yield token

# Map: lowercase base form -> set of observed case variants
case_variants_by_base = defaultdict(set)

# Track the best (most variants) incrementally
best_base = None
best_variants = set()
best_count = 0

with open(DATA_DIR / "tweets.json", "r", encoding="utf-8") as fh:
    for line in fh:
        tweet = json.loads(line)
        text = tweet.get("text", "")

        for original in iter_alpha_tokens_preserve_case(text):
            base = original.lower()

            # Skip very common stopwords (for cleaner results)
            if base in STOPWORDS:
                continue

            variants_set = case_variants_by_base[base]
            before = len(variants_set)
            variants_set.add(original)
            after = len(variants_set)

            # Update winner only when a new distinct variant is added
            if after > before and after > best_count:
                best_count = after
                best_base = base
                best_variants = variants_set

# Clean, readable output
sorted_variants = sorted(best_variants, key=lambda s: (s.lower(), s))

print(f"Word with most case variations (base form): '{best_base}'")
print(f"Number of distinct case variations: {best_count}")
print("Variations (unique):")
print(", ".join(sorted_variants))


Word with most case variations (base form): 'this'
Number of distinct case variations: 11
Variations (unique):
THIS, THiS, THis, ThIs, ThiS, This, tHIS, tHiS, thIS, thiS, this


## 6) Twitter: Longest dictionary-word sequence

### Objective
Identify the longest contiguous sequence of dictionary words appearing in the Twitter dataset that occurs in at least 10 distinct tweets.

A dictionary word is defined as a token present in the provided English word list. Sequences are formed by grouping adjacent dictionary words within a tweet.

For example, given the tweet  
“word1 word2 word3 word4 word5 word6 word7”,  
and a dictionary containing “word1”, “word3”, “word4”, and “word7”,  
the resulting dictionary-word sequences are:
- “word1”
- “word3 word4”
- “word7”

In this example, the longest valid sequence is “word3 word4”. Only sequences meeting the minimum frequency threshold across the dataset are considered.


In [8]:
# 6) Twitter: Longest dictionary-word sequence (>= 10 tweets)

from collections import Counter

def load_dictionary_words(path=None) -> set[str]:
    """Load dictionary words into a lowercase set for O(1) membership checks."""
    if path is None:
        path = DATA_DIR / "linuxwords"

    with open(path, "r", encoding="utf-8") as fh:
        return {line.strip().lower() for line in fh if line.strip()}

def iter_dictionary_runs(tokens: list[str], dict_words: set[str]):
    """
    Yield maximal contiguous runs of dictionary words as tuples.
    Example: tokens -> ["a","b","c"], where a,c in dict but b not -> yields ("a",), ("c",)
    """
    run = []
    for tok in tokens:
        if tok in dict_words:
            run.append(tok)
        else:
            if run:
                yield tuple(run)
                run = []
    if run:
        yield tuple(run)

dict_words = load_dictionary_words()
run_counts = Counter()  # run tuple -> number of distinct tweets it appears in

with open(DATA_DIR / "tweets.json", "r", encoding="utf-8") as fh:
    for line in fh:
        tweet = json.loads(line)
        text = tweet.get("text", "")

        # Lowercase + punctuation stripped (OK here; we are matching dictionary words)
        toks = normalise(text)

        # Count each run at most once per tweet
        seen_runs = set(iter_dictionary_runs(toks, dict_words))
        run_counts.update(seen_runs)

# Find the longest run that appears in at least 10 tweets
min_tweets = 10
best_run = ()
best_count = 0

for run, c in run_counts.items():
    if c >= min_tweets:
        if len(run) > len(best_run) or (len(run) == len(best_run) and c > best_count):
            best_run = run
            best_count = c

best_text = " ".join(best_run)

print(f"Longest dictionary-word sequence (>= {min_tweets} tweets):")
print(best_text)
print(f"Length (words): {len(best_run)}")
print(f"Distinct tweets: {best_count}")


Longest dictionary-word sequence (>= 10 tweets):
bonus fact fake friends and fans will turn on you at the very first rumor of drama the real will need tons of proof and will defend you
Length (words): 28
Distinct tweets: 17


## 7) Matching tweets to Wikipedia articles using Jaccard similarity


### Objective
Given a tweet, identify the Wikipedia article whose body text is most similar to the tweet content, using the Jaccard similarity index as the similarity metric.

Similarity is computed by representing both the tweet and each article as sets of unique tokens (derived from article body text only), and selecting the article with the highest Jaccard score.

The Jaccard index between two texts A and B is defined as:

$J(A, B) = \frac{|A \cap B|}{|A \cup B|}$

For example, given:
- A: “python is a fun programming language”
- B: “programming with python is always fun”

The intersection contains {python, is, fun, programming} and the union contains 8 unique tokens, yielding a Jaccard similarity of 0.5.

This approach provides a simple and interpretable measure of lexical overlap, though it can be sensitive to differences in text length.

In [10]:
# 7) Matching tweets to Wikipedia articles using Jaccard similarity

def to_word_set(text: str) -> set[str]:
    """Convert text to a set of unique normalised tokens."""
    return set(normalise(text))

def jaccard_index(a: set[str], b: set[str]) -> float:
    """Compute J(A,B) = |A ∩ B| / |A ∪ B| efficiently."""
    if not a and not b:
        return 0.0
    # Iterate over smaller set for fewer membership checks
    if len(a) > len(b):
        a, b = b, a
    intersection = sum(1 for tok in a if tok in b)
    union = len(a) + len(b) - intersection
    return intersection / union if union else 0.0

def best_match(tweet_id: str) -> tuple[int | None, float]:
    """
    Given a tweet ID, return (best_article_id, best_jaccard_score)
    comparing the tweet text against Wikipedia article body text only.
    """
    # 1) Locate the tweet and build its word set
    tweet_terms = None
    with open(DATA_DIR / "tweets.json", "r", encoding="utf-8") as fh:
        for line in fh:
            tweet = json.loads(line)
            if tweet.get("id_str") == tweet_id:
                tweet_terms = to_word_set(tweet.get("text", ""))
                break

    if tweet_terms is None:
        return None, 0.0

    tweet_len = len(tweet_terms)

    # 2) Stream Wikipedia articles and keep the running best match
    best_article_id = None
    best_similarity = -1.0

    with open(DATA_DIR / "wiki-articles.json", "r", encoding="utf-8") as fh:
        for line in fh:
            article = json.loads(line)
            article_terms = to_word_set(article.get("text", ""))
            article_len = len(article_terms)

            # Upper bound on Jaccard given set sizes:
            # |A∩B| <= min(|A|,|B|) and |A∪B| >= max(|A|,|B|)
            # => J <= min(|A|,|B|) / max(|A|,|B|)
            smaller = tweet_len if tweet_len <= article_len else article_len
            larger = article_len if tweet_len <= article_len else tweet_len
            jaccard_upper_bound = (smaller / larger) if larger else 0.0

            if jaccard_upper_bound <= best_similarity:
                continue  # prune weak candidates

            similarity = jaccard_index(tweet_terms, article_terms)

            if similarity > best_similarity:
                best_similarity = similarity
                best_article_id = article.get("id")

    return best_article_id, best_similarity

# Demo call 
tweet_id = "1380411674478354434"
article_id, score = best_match(tweet_id)

if article_id is None:
    print(f"No tweet found for id={tweet_id}")
else:
    print(f"Best match for tweet {tweet_id}: article {article_id} (Jaccard={score:.3f})")


Best match for tweet 1380411674478354434: article 32199 (Jaccard=0.115)


## 8) Inverted index and wildcard search

### Objective
Build a mini search capability across both corpora (Wikipedia articles + tweets) by creating a shared **positional inverted index**, then use it to support **wildcard-style phrase queries**.

---

### Part 1: Positional inverted index
To enable efficient retrieval without scanning every document, I construct an inverted index that maps each token to:
- the source type (Wikipedia article or tweet)
- the document ID
- the token position(s) within that document

Example (toy):
If an article has ID 7 and text “python coding is so much fun”, then:
- `coding` appears at position 1 in article 7
- `fun` appears at position 5 in article 7

This positional information enables phrase and pattern matching rather than simple keyword lookup.

---

### Part 2: Wildcard search over the index
Using the positional index, I implement wildcard-style queries where `*` represents any single token.

Example query:
`word1 * * * word2 * word3`

This matches documents where:
- `word1` is followed by any 3 tokens,
- then `word2`,
- then any 1 token,
- then `word3`.

For this notebook, queries are assumed to start and end with a word (not `*`).


In [11]:
# 8) Inverted index and wildcard search

# -------------------------
# Part 1: Positional inverted index
# index[token][source_label][doc_id] -> list of positions
# -------------------------

index = defaultdict(lambda: {"article": defaultdict(list), "tweet": defaultdict(list)})

def add_document_to_index(source_label: str, document_id, text: str) -> None:
    """Add token positions for a single document to the shared index."""
    tokens = normalise(text)  # lowercase + punctuation stripped (matches index keys)
    postings_for_token = index  # local alias for speed in tight loop

    for pos, tok in enumerate(tokens):
        postings_for_token[tok][source_label][document_id].append(pos)

# Build index: Wikipedia articles
with open(DATA_DIR / "wiki-articles.json", "r", encoding="utf-8") as fh:
    for line in fh:
        article = json.loads(line)
        add_document_to_index("article", article.get("id"), article.get("text", ""))

# Build index: Tweets
with open(DATA_DIR / "tweets.json", "r", encoding="utf-8") as fh:
    for line in fh:
        tweet = json.loads(line)
        add_document_to_index("tweet", tweet.get("id_str"), tweet.get("text", ""))

# -------------------------
# Part 2: Wildcard search over the index
# -------------------------

def parse_query(query_text: str) -> tuple[list[str], list[int]]:
    """
    Convert a wildcard query into:
    - words: list of normalised query terms (excluding '*')
    - gaps:  list where gaps[i] is the number of '*' between words[i] and words[i+1]
    Assumes query starts and ends with a word (not '*').
    """
    raw_tokens = query_text.split()

    cleaned = []
    for t in raw_tokens:
        if t == "*":
            cleaned.append("*")
        else:
            # normalise while preserving '*' (we don't expect '*' inside tokens)
            tok = t.translate(TRANSLATOR).lower()
            if tok:
                cleaned.append(tok)

    words = []
    gaps = []
    gap = 0

    for t in cleaned:
        if t == "*":
            gap += 1
        else:
            if words:
                gaps.append(gap)
            words.append(t)
            gap = 0

    return words, gaps

def get_candidate_documents(words: list[str], source_label: str) -> set:
    """
    Candidate docs must contain ALL query words.
    We intersect doc-id sets per word, starting from the smallest set.
    """
    doc_sets = []
    for w in words:
        postings = index.get(w)
        if not postings:
            return set()
        docs = set(postings[source_label].keys())
        if not docs:
            return set()
        doc_sets.append(docs)

    doc_sets.sort(key=len)
    candidates = doc_sets[0]
    for s in doc_sets[1:]:
        candidates &= s
        if not candidates:
            break
    return candidates

def document_matches_pattern(source_label: str, document_id, words: list[str], gaps: list[int]) -> bool:
    """
    Positional match for patterns like:
    word1 * * word2 * word3
    where each '*' represents exactly one token.
    """
    # Fetch position lists; if any term missing in this doc, fail fast
    position_lists = []
    for w in words:
        positions = index[w][source_label].get(document_id)
        if not positions:
            return False
        position_lists.append(positions)

    # Try each occurrence of the first word as a start
    first_positions = position_lists[0]
    for start_pos in first_positions:
        cur_pos = start_pos
        ok = True

        for i in range(1, len(position_lists)):
            expected = cur_pos + gaps[i - 1] + 1
            next_positions = position_lists[i]

            # Fast fail if expected is beyond last occurrence
            if expected > next_positions[-1]:
                ok = False
                break

            # Binary search would be ideal, but list lengths are usually small.
            # Use a pointer scan to find expected.
            j = 0
            while j < len(next_positions) and next_positions[j] < expected:
                j += 1

            if j == len(next_positions) or next_positions[j] != expected:
                ok = False
                break

            cur_pos = expected

        if ok:
            return True

    return False

def wildcard_search(query_text: str) -> dict:
    words, gaps = parse_query(query_text)
    if not words:
        return {"article": [], "tweet": []}

    candidate_articles = get_candidate_documents(words, "article")
    candidate_tweets = get_candidate_documents(words, "tweet")

    matched_articles = [doc_id for doc_id in candidate_articles
                        if document_matches_pattern("article", doc_id, words, gaps)]
    matched_tweets = [doc_id for doc_id in candidate_tweets
                      if document_matches_pattern("tweet", doc_id, words, gaps)]

    matched_articles.sort()
    matched_tweets.sort()

    return {"article": matched_articles, "tweet": matched_tweets}

def print_search_results(query: str, results: dict, preview_n: int = 10) -> None:
    """Skimmable output: counts + a small preview."""
    a = results["article"]
    t = results["tweet"]

    print(f"Query: {query}")
    print(f"Articles matched: {len(a)}")
    if a:
        print("Article IDs (preview):", a[:preview_n], "..." if len(a) > preview_n else "")

    print(f"Tweets matched: {len(t)}")
    if t:
        print("Tweet IDs (preview):", t[:preview_n], "..." if len(t) > preview_n else "")
    print("-" * 60)

# Demo queries (clean output)
print_search_results("new * uk", wildcard_search("new * uk"))
print_search_results("institute * * * university * * university * university",
                     wildcard_search("institute * * * university * * university * university"))


Query: new * uk
Articles matched: 2
Article IDs (preview): ['30028', '32356'] 
Tweets matched: 14
Tweet IDs (preview): ['1380448072661106690', '1380458625508999172', '1380458856187318275', '1380460043200565249', '1380463180552544256', '1380471913076625410', '1380474505152368648', '1380476241594224640', '1380484873455042560', '1380494818187563009'] ...
------------------------------------------------------------
Query: institute * * * university * * university * university
Articles matched: 1
Article IDs (preview): ['18879'] 
Tweets matched: 0
------------------------------------------------------------


## Summary and future directions

This notebook demonstrated how large text corpora can be processed efficiently using core Python only, with an emphasis on streaming I/O, careful token normalisation, and appropriate use of built-in data structures.

Key themes included:
- Streaming-based processing of large, line-delimited JSON datasets to minimise memory usage.
- Use of interpretable similarity measures (Jaccard index) and simple pruning strategies to reduce unnecessary computation.
- Construction of a shared positional inverted index to support efficient phrase and wildcard-style search across corpora.

Possible extensions include:
- More robust tokenisation for social media text (e.g. URLs, emojis, hashtags).
- Alternative similarity measures such as TF–IDF with cosine similarity.
- Persisting the inverted index to disk for larger-than-memory datasets.
